In [ ]:
# ==============================================
# セル1: 要約とFAQを作成し、それぞれ別のWord文書に保存
# ==============================================

!pip install python-docx openai==0.27.0 pinecone-client==2.2.1 python-dotenv --quiet

import os
import re
import openai
import pinecone
import docx  # python-docx
from docx import Document
from google.colab import files
from IPython.display import display, Markdown

# (A) APIキー読み込み
def load_api_keys(txt_filepath="api_keys.txt"):
    if not os.path.exists(txt_filepath):
        raise FileNotFoundError(f"[ERROR] {txt_filepath} が見つかりません。")
    with open(txt_filepath, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line or line.startswith("#"):
                continue
            if "=" in line:
                k,v=line.split("=",1)
                os.environ[k.strip()] = v.strip()

load_api_keys("api_keys.txt")
openai.api_key = os.getenv("OPENAI_API_KEY", "")

# (B) 既存のWord文書をアップロードし、テキストを抽出
print("既存のWord文書(.docx)をアップロードしてください。")
uploaded = files.upload()

if not uploaded:
    print("アップロードがキャンセルされました。")
else:
    docx_file = list(uploaded.keys())[0]
    print("アップロード完了:", docx_file)

    # docx読み込み
    original_doc = docx.Document(docx_file)

    # 全テキスト抽出
    paragraphs = [p.text for p in original_doc.paragraphs]
    doc_text = "\n".join(paragraphs).strip()
    doc_title = os.path.splitext(docx_file)[0]
    print(f"Word文書タイトル: {doc_title}")
    print(f"テキスト総字数: {len(doc_text)}")

    # (C) ChatGPTに要約とFAQを生成させる
    def generate_summary_and_faq(document_text, title="Untitled", num_faq=5):
        # 要約プロンプト
        summary_prompt = f"""
        以下のドキュメント（タイトル: {title}）を要約してください。

        【要件】:
          1. アクセス権限や冒頭の枠情報など、細かい権限制限の話は最小限にしてください。
          2. もしドキュメントに複数パターン（例: 会社・勘定科目など）が出てくる場合は、各パターンの特徴や事例をある程度詳しく書いてください。
          3. 設定や作業手順（新規追加、編集、削除など）があれば、具体的な操作フローや注意点（文字数制限、コードの一意性 など）を順序立てて整理してください。
          4. 重複説明や別ガイド参照が必要な箇所は深追いしなくて構いません。基本的には「ここでは簡潔に」としてください。
          5. 全体的に、箇条書きや段階的な説明を使い、要点が分かりやすいようにまとめてください。長すぎる冗長表現は避けてください。
          6. 文字数の目安は、A4で1～2ページ程度までに収めるイメージでお願いします。

        --ドキュメント--
        {document_text}
        """

        # FAQプロンプト
        faq_prompt = f"""
        以下のドキュメント(タイトル:{title})について、
        ユーザがよく聞きそうな質問を最低でも5件、最大7件ほど想定し、
       「Q1: ... / A1: ...」形式で作成してください。

        【必須要件】:
        - ドキュメントが短い場合でも、一般的な運用上の疑問を補って質問を作成し、すべてに回答してください。
        - アクセス権限など細かい制限が繰り返される場合は1問程度で簡潔に済ませてください。
        - 残りのQ&Aは、設定時の注意点、操作方法、メンテナンス（更新・削除）、トラブルシュートなどを想定してください。
        - QとAのフォーマットは必ず「Qx:」「Ax:」のペアで書いてください。

        --ドキュメント--
        {document_text}
        """

        # 要約生成
        summary_result = ""
        try:
            resp_sum = openai.ChatCompletion.create(
                model="gpt-4o",  # GPT-4の独自モデルID
                messages=[
                    {"role": "system", "content": "あなたは優秀なアシスタントです。"},
                    {"role": "user", "content": summary_prompt},
                ],
                temperature=0
            )
            summary_result = resp_sum["choices"][0]["message"]["content"]
        except Exception as e:
            print("要約生成エラー:", e)
            summary_result = "要約生成に失敗しました。"

        # FAQ生成
        faq_result_list = []
        try:
            resp_faq = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "あなたは優秀なアシスタントです。"},
                    {"role": "user", "content": faq_prompt},
                ],
                temperature=0
            )
            faq_raw = resp_faq["choices"][0]["message"]["content"]
            # 簡易パース: Q1: ... / Q2: ... などを区切る
            lines = re.split(r"(Q\d+[:：])", faq_raw)
            tmp_q = ""
            tmp_a = ""
            for i in range(len(lines)):
                part = lines[i].strip()
                if re.match(r"^Q\d+[:：]", part):
                    tmp_q = part
                    tmp_a = ""
                else:
                    if tmp_q:
                        tmp_a = part
                        faq_result_list.append({"q": tmp_q, "a": tmp_a})
                        tmp_q = ""
                        tmp_a = ""
        except Exception as e:
            print("FAQ生成エラー:", e)
            faq_result_list = []

        return {
            "summary": summary_result.strip(),
            "faq": faq_result_list
        }

    # 生成実行
    result = generate_summary_and_faq(doc_text, title=doc_title, num_faq=5)
    summary_text = result["summary"]
    faq_list = result["faq"]

    # 生成結果を表示
    print("\n[=== 要約 ===]\n")
    display(Markdown(summary_text))

    print("\n[=== FAQ ===]\n")
    for faq_item in faq_list:
        display(Markdown(f"**{faq_item['q']}**\n{faq_item['a']}"))

    # (D) 別々のWord文書を作成し、要約とFAQを分割保存
    # --- Summary Doc ---
    summary_doc_title = doc_title + "_Summary.docx"
    doc_summary = docx.Document()
    doc_summary.add_heading(f"{doc_title} - 要約", level=1)

    for line in summary_text.split("\n"):
        doc_summary.add_paragraph(line)

    doc_summary.save(summary_doc_title)
    print(f"\n[INFO] 要約のWord文書を作成しました: {summary_doc_title}")

    files.download(summary_doc_title)

    # --- FAQ Doc ---
    faq_doc_title = doc_title + "_FAQ.docx"
    doc_faq = docx.Document()
    doc_faq.add_heading(f"{doc_title} - FAQ", level=1)

    for i, faq_item in enumerate(faq_list, start=1):
        q_par = doc_faq.add_paragraph(style='List Number')
        q_par.add_run(faq_item['q']).bold = True
        doc_faq.add_paragraph(faq_item['a'], style='List Bullet')

    doc_faq.save(faq_doc_title)
    print(f"[INFO] FAQのWord文書を作成しました: {faq_doc_title}")

    files.download(faq_doc_title)

    print("\n(★ダウンロードしてローカルに保管し、後日Pinecone登録などに利用してください)")


既存のWord文書(.docx)をアップロードしてください。


Saving Exp_SG_Ledgers-jp.docx to Exp_SG_Ledgers-jp.docx
アップロード完了: Exp_SG_Ledgers-jp.docx
Word文書タイトル: Exp_SG_Ledgers-jp
テキスト総字数: 4056

[=== 要約 ===]



### Exp_SG_Ledgers-jp 要約

#### 概要
- **元帳の定義**: 元帳は、経費精算の会計システムとの接点であり、取引が個別の勘定科目にリストされる最終入力の会計帳簿です。
- **ユーザーのカテゴリー**:
  - **経理担当者**: 経費精算データを使用して支払システムで取引を作成。
  - **勘定科目管理者**: 元帳や勘定科目構造の定義を担当。

#### 総勘定元帳への対応付け
- **取引の「担当者」**: 経費が適用される業務分野を定義。部門やプロジェクトなどの会計セグメントで構成。
- **取引の「対象」**: 勘定科目表に関連し、財務および税のレポート用に分類。

#### 複数の会社に関する対応付け
- **会社の定義**: 貸借勘定科目を管理する事業体。親会社の下に複数の会社が存在することも。
- **勘定科目コードの組み合わせ**:
  1. 個別の会計システムを使用。
  2. 会計システムを共有。
  3. 一部共有し、固有のコードも定義。
  4. 異なるデータフィールドを使用。

#### 元帳に関する考慮事項
- **元帳の定義**: Concur システムで1つ以上の元帳を定義可能。各元帳は単一の勘定科目構造を持つ。
- **注意点**: 勘定科目コードの階層変更は構成を無効にする可能性。

#### 作業手順
- **元帳へのアクセス**:
  1. [管理] ＞ [経費精算] に進む。
  2. [勘定科目管理] をクリックし、[勘定科目構造] タブを選択。

- **新しい元帳の追加**:
  1. [勘定科目構造] タブの [新規] をクリック。
  2. 必須情報を入力し、[保存]。

- **既存の元帳の編集**:
  1. [勘定科目構造] タブで編集するフィールドをクリック。
  2. 情報を編集し、[保存]。

- **既存の元帳の削除**:
  1. [勘定科目構造] タブで目的の元帳の名前をクリック。
  2. [削除] をクリックし、[はい] で削除を確定。

この要約は、元帳の管理に関する基本的な操作フローと注意点を簡潔にまとめています。詳細な設定や特定のガイドラインについては、関連するユーザーガイドを参照してください。


[=== FAQ ===]



**Q1:**
元帳へのアクセス権限がない場合、どのように対処すればよいですか？
A1: アクセス権限がない場合は、社内のConcur管理者に連絡して、適切な権限を付与してもらう必要があります。管理者が対応できない場合は、Concurクライアントサポートにサービスリクエストを行ってください。

...

<IPython.core.display.Javascript object>

[INFO] FAQのWord文書を作成しました: Exp_SG_Ledgers-jp_FAQ.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


(★ダウンロードしてローカルに保管し、後日Pinecone登録などに利用してください)
